In [4]:
import gurobipy as gp
from gurobipy import GRB

# The Model

In [5]:
theater_seat_organizer = gp.Model()

Academic license - for non-commercial use only - expires 2022-01-02
Using license file /Users/nsawan/gurobi.lic


# The Parameters

In [6]:
# Uncomment the single file that will be tested and recomment the previous case

test_case = "testcase1.txt"
#test_case = "testcase2.txt"
#test_case = "testcase3.txt"
#test_case = "testcase4.txt"


f = open(test_case,"r")
testfile = f.readlines()

test_name = testfile[0].split("=")[1]

string_num_of_customers_in_group = testfile[1].split(" = ")[1]

#Convert list of string to list of int
number_of_customers_in_group_list = string_num_of_customers_in_group.strip("[").strip("\n").strip("]").split(",")
map_list = map(int, number_of_customers_in_group_list)
number_of_customers_in_group = list(map_list)


number_of_seats = int(testfile[2].split(" = ")[1])

group_exclusion_cost = float(testfile[3].split(" = ")[1])

# Decision Variables

In [7]:
person_inGroup_atSeat = {}

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            
            person_inGroup_atSeat[customer_index, group_num, seat] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_atSeat" + str(seat))

groupAccepted = {}
for group_num in range(len(number_of_customers_in_group)):
    groupAccepted[group_num] = theater_seat_organizer.addVar(vtype = GRB.BINARY, name = "group" + str(group_num) + "_admitted")


            

# The Model

In [8]:
objectiveFunction = gp.LinExpr()

for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            objectiveFunction += person_inGroup_atSeat[customer_index, group_num, seat]   
    
for group_num in range(len(number_of_customers_in_group)):
    objectiveFunction -= group_exclusion_cost * (1-groupAccepted[group_num])

theater_seat_organizer.setObjective(objectiveFunction,  GRB.MAXIMIZE)

# Constraints

In [9]:
# at most one person per seat
for seat in range(number_of_seats):
    at_most_one_person_per_seat = gp.LinExpr()
    
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            at_most_one_person_per_seat += person_inGroup_atSeat[customer_index, group_num, seat]   
    theater_seat_organizer.addConstr(at_most_one_person_per_seat <= 1, name = "at_most_one_person_at_seat" + str(seat))



In [10]:
# the whole group must be added
for group_num in range(len(number_of_customers_in_group)):
    
    group_must_be_added = gp.LinExpr() # collects the sum of all the seats the group sits in
    is_group_accepted = gp.LinExpr(number_of_customers_in_group[group_num] * groupAccepted[group_num]) #determines if the group will be seated or not
    
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        for seat in range(number_of_seats):
            group_must_be_added += person_inGroup_atSeat[customer_index, group_num, seat]   
    
    theater_seat_organizer.addConstr(group_must_be_added == is_group_accepted, name = "is_group_" + str(group_num) + "_accepted")


In [11]:
# each person should be assigned to only one seat 
for group_num in range(len(number_of_customers_in_group)):
    for customer_index in range(0, number_of_customers_in_group[group_num]):
        seats_per_person = gp.LinExpr()
        
        for seat in range(number_of_seats):
            seats_per_person += person_inGroup_atSeat[customer_index, group_num, seat] 
            
        theater_seat_organizer.addConstr(seats_per_person <= 1, name = "person" + str(customer_index) + "_group" + str(group_num) + "_uniqueSeat")


In [12]:
# There must be an empty seat between groups 
for seat in range(number_of_seats - 1):
    
    for group_num in range(len(number_of_customers_in_group)):
        for other_group_num in range(len(number_of_customers_in_group)):
            
            if not(group_num == other_group_num):    
                
                for customer_index in range(0, number_of_customers_in_group[group_num]):
                    for customer2_index in range(0, number_of_customers_in_group[other_group_num]):
                        
                        seat1 = person_inGroup_atSeat[customer_index, group_num, seat]
                        seat2 = person_inGroup_atSeat[customer2_index, other_group_num, seat+1]
                        label = "person" + str(customer_index) + "_inGroup" + str(group_num) + "_andPerson" + str(customer2_index) + "+inGroup" + str(other_group_num) + "_cannotSitTogether"
                        theater_seat_organizer.addConstr( seat1 + seat2 <= 1, name = label)

# Optimize 

In [13]:
theater_seat_organizer.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 6241 rows, 703 columns and 14451 nonzeros
Model fingerprint: 0xfbb55a79
Variable types: 0 continuous, 703 integer (703 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -9.0000000
Presolve removed 5638 rows and 0 columns
Presolve time: 0.05s
Presolved: 603 rows, 703 columns, 8253 nonzeros
Variable types: 0 continuous, 703 integer (703 binary)

Root relaxation: objective 1.400000e+01, 277 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      14.0000000   14.00000  0.00%     -    0s

Explored 0 nodes (277 simplex iterations) in 0.

# Post Process

In [14]:
seating_scheme = list()
for seat in range(number_of_seats):
    seatFound = False
    for group_num in range(len(number_of_customers_in_group)):
        for customer_index in range(0, number_of_customers_in_group[group_num]):
            result = person_inGroup_atSeat[customer_index, group_num, seat]
            
            if( result.x == 1 ):
                seatFound = True
                print( "Seat" + str(seat) + ": person" + str(customer_index) + " fromGroup" + str(group_num))
                
                seating_scheme.append((customer_index, group_num))
    
    if(not seatFound):
        print("Seat" + str(seat) + ": <empty>")
        seating_scheme.append(None)

Seat0: <empty>
Seat1: <empty>
Seat2: person0 fromGroup1
Seat3: <empty>
Seat4: person1 fromGroup2
Seat5: <empty>
Seat6: <empty>
Seat7: <empty>
Seat8: <empty>
Seat9: <empty>
Seat10: <empty>
Seat11: <empty>
Seat12: <empty>
Seat13: <empty>
Seat14: <empty>
Seat15: <empty>
Seat16: <empty>
Seat17: person0 fromGroup0
Seat18: <empty>
Seat19: <empty>
Seat20: person2 fromGroup0
Seat21: <empty>
Seat22: <empty>
Seat23: person5 fromGroup2
Seat24: <empty>
Seat25: <empty>
Seat26: person4 fromGroup2
Seat27: <empty>
Seat28: person0 fromGroup2
Seat29: person2 fromGroup2
Seat30: <empty>
Seat31: person2 fromGroup1
Seat32: <empty>
Seat33: <empty>
Seat34: <empty>
Seat35: <empty>
Seat36: person3 fromGroup1
Seat37: person1 fromGroup1
Seat38: <empty>
Seat39: <empty>
Seat40: <empty>
Seat41: person3 fromGroup2
Seat42: <empty>
Seat43: <empty>
Seat44: <empty>
Seat45: person4 fromGroup1
Seat46: <empty>
Seat47: <empty>
Seat48: <empty>
Seat49: person1 fromGroup0


In [15]:
genericSpacing = list()
last_group = -1
for value in seating_scheme:
    if(type(value) == tuple and value[1] != last_group):
        genericSpacing.append((None,None))
        genericSpacing.append(value)
        last_group = value[1]
        
    
    elif(type(value) == tuple and value[1] == last_group):
        genericSpacing.append(value)
        
genericSpacing.pop(0)

while(genericSpacing[0][1] == genericSpacing[-1][1]):
    genericSpacing.append(genericSpacing[0])
    genericSpacing.pop(0)
    
genericSpacing.pop(0)
    
for members in genericSpacing:
    if not (members[0] == None):
        print("person " + str(members[0] + 1) + " from group " + str(members[1] + 1))
    else:
        print("<SPACE>")



<SPACE>
person 2 from group 3
<SPACE>
person 1 from group 1
person 3 from group 1
<SPACE>
person 6 from group 3
person 5 from group 3
person 1 from group 3
person 3 from group 3
<SPACE>
person 3 from group 2
person 4 from group 2
person 2 from group 2
<SPACE>
person 4 from group 3
<SPACE>
person 5 from group 2
<SPACE>
person 2 from group 1


In [134]:
for group_num in range(len(groupAccepted)):
    if(groupAccepted[group_num].x != 1):
        print("Group " + str(group_num + 1) + " is excluded")
    else:
        print("Group " + str(group_num + 1) + " is accepted")
#print(groupAccepted)

Group 1 is accepted
Group 2 is accepted
Group 3 is accepted
